In [ ]:
import primer3

# Import modules
from multiplexdesigner.primer3_port import seqtm

In [2]:
sequence = 'CTGATGGGACCCACTCCATCG'

In [3]:
seqtm(sequence)

Running oligotm


TmResult(Tm=62.24775924663061, bound=69.63903765565786)

In [4]:
oligo_calc = primer3.thermoanalysis.ThermoAnalysis()

oligo_calc.set_thermo_args(
    mv_conc = 50.0,
    dv_conc = 1.5,
    dntp_conc = 0.6,
    dna_conc = 50.0,
    dmso_conc = 0.0,
    dmso_fact = 0.6,
    formamide_conc = 0.0,
    annealing_temp_c = -10,
    temp_c = 37.0,
    max_nn_length = 60,
    max_loop = 30,
    tm_method = 'santalucia',
    salt_corrections_method = 'santalucia'
)

oligo_calc.calc_tm(sequence)

62.24775924663061